In [1]:
import numpy as np
import os
import dill as pickle
import matplotlib.pyplot as plt
import pandas as pd
import sys
import hardware_control.wx_programs as wx
import hardware_control.bnc as bnc
from classes.generator import *
from hardware_control.hardware_config import *
from experiment_configuration.values import *
from classes.qubit_class import *
import daq.daq_programs_homo as daq
import classifiers.classifier as classifier
import seaborn as sns
import standard_sequences.reg_tomo as tomo

In [2]:
q1 = Qubit(q1_dict, readout_dict)
q2 = Qubit(q2_dict, readout_dict)
readout = Readout(readout_dict)
print(f"{q1}\n{q2}")

wx_addr = wx.get_wx_address()
main_directory = r"C:\Users\quantum1\Documents"
save_dir = rf"{main_directory}\Python Scripts\Important Blue Fridge Python Files\New\nonlinear_QM\data"
target_bnc_address_6 = "USB0::0x03EB::0xAFFF::411-433500000-0753::INSTR"
bnc.set_bnc_output(
    general_vals_dict["qubit_bnc"], power_dBm=13, bnc_addr=target_bnc_address_6
)
bnc.set_bnc_output(
    readout_dict["RO_LO"],
    power_dBm=readout_dict["RO_LO_pwr"],
    bnc_addr=bnc_address["target_bnc_black"],
)
bnc.set_bnc_output(
    general_vals_dict["TWPA_freq"],
    general_vals_dict["TWPA_pwr"],
    bnc_addr=bnc_address["big_agilent"],
)

Qubit(ro_amp=1, ROIF=0.10275000000000034, ge_ssm=-0.1145, ro_freq=6.72745, qubit_thr=[-10000, -600], ef_ssm=-0.2568, ge_amp=0.8, ef_amp=1.5, qubit_id=q1, RO_LO=6.6247, ge_time=81.17534608287957, RO_LO_pwr=16, IQ_angle=205, RO_IF=None, ro_dur=4000, ef_time=49.13671850689116)
Qubit(ro_amp=1.5, ROIF=0.030899999999999928, ge_ssm=-0.154, ro_freq=6.6556, qubit_thr=[-10000, 1900], ef_ssm=-0.2962, ge_amp=0.8, ef_amp=1, qubit_id=q2, RO_LO=6.6247, ge_time=45.047198597262124, RO_LO_pwr=16, IQ_angle=85, RO_IF=None, ro_dur=4000, ef_time=None)


In [3]:
def run_tomo(
    q1: object,
    q2: object,
    general_vals_dict: dict,
    copief: int,
    copige: int,
    swap_freq: float,
    swap_time: float,
    reps: int,
    tomography: str,
):
    """
    Runs a single instance of the nonhermitian ef Rabi experiment (with an e-swap to Q2)
    and processes the resulting IQ data to compute probabilities vs. time.

    Returns:
        df_prob (pd.DataFrame): A DataFrame whose index is the time (computed as
                                np.linspace(0, sweep_time/1000, num_steps)) and which has
                                columns ['P_f', 'P_e', 'P_g'] corresponding to the probabilities
                                of the f, e, and g states respectively.
    """
    coswap = 1
    # Run the experiment
    tomo. gef_tomo(
    q1,
    q2,
    general_vals_dict,
    copief,
    coswap,
    copige,
    num_steps=3,
    swap_freq=swap_freq,
    swap_time=swap_time,
    phase=0,
    swap_amp=1,
    tomo_comp=tomography,
    buffer=0)

    wx.wx_set_and_amplitude_and_offset(
        amp=general_vals_dict["wx_amps"], offset=general_vals_dict["wx_offs"]
    )
    # Acquire the raw IQ data
    values = daq.run_daq_het_2q(
        q1, q2, num_patterns=3, num_records_per_pattern=reps, verbose=False
    )

    # Retrieve raw IQ data from the acquired values
    I1_raw = values.rec_readout_1[0]
    Q1_raw = values.rec_readout_1[1]
    I2_raw = values.rec_readout_2[0]
    Q2_raw = values.rec_readout_2[1]

    # Build a DataFrame from the IQ data for classification
    IQ_df = pd.DataFrame({"I1": I1_raw, "Q1": Q1_raw, "I2": I2_raw, "Q2": Q2_raw})

    # Classify the IQ data (classifier.classify returns a DataFrame that has a 'predicted' column)
    classified = classifier.classify(IQ_df)
    states = classified["predicted"]
    # Reshape the predicted states.
    # (Assume that classifier.reshape_for_exp returns an array of shape (num_steps, reps),
    #  where each row corresponds to a time step and holds all the state measurements for that step.)
    states_reshaped = classifier.reshape_for_exp(states, reps, 3)
    probabilties = classifier.probabilities(states_reshaped)
    population = classifier.population(states_reshaped)

    # Compute time values as the index for the DataFrame.
    #times = np.linspace(0, sweep_time / 1000, 3)

    # Build a DataFrame that holds the probability vs. time.
    df_prob = pd.DataFrame(
        {
           # "time": times,
            "P_f": probabilties["P_f"],
            "P_e": probabilties["P_e"],
            "P_g": probabilties["P_g"],
        },
    )

    return df_prob, values

In [4]:
reps = 1000
sweep_time = 10
swap_freq = -0.0195
swap_time = 0.5 * 6 / abs(swap_freq)
J = 0.1
tomography = "y"
df_prob, values = run_tomo(
    q1,
    q2,
    general_vals_dict,
    copief = 0,
    copige = 1,
    swap_freq = swap_freq,
    swap_time = swap_time,
    reps = reps,
    tomography = tomography,
)

writing to C:\arbsequences\strong_dispersive_withPython\test_pulse_ringupdown_bin\
writing ch1
writing ch2
writing ch3
writing ch4
loading C:\arbsequences\strong_dispersive_withPython\test_pulse_ringupdown_bin\
num_steps 3
loading ch1
loading ch2
loading ch3
loading ch4


KeyboardInterrupt: 

In [ ]:
df_prob

,P_f,P_e,P_g
0,0.437,0.165,0.398
1,0.448,0.148,0.404
2,0.427,0.144,0.429
